The main goal of [Delta E (Δ*E*)](https://en.wikipedia.org/wiki/Color_difference#Delta_E) was seemingly to test whether two solid color patches would be perceived as equivalent colors. For instance, if a tractor body is composed of "green" parts from two separate factories, will those parts appear to be the same green? Smaller values of Delta E suggest a higher probability that a human would say that the colors are the same.

However, Delta E is not necessarily a good measure of contrast. Very large values (much greater than 6) might not be in the order you might expect. As demoed below, the CIELAB 1976 Delta E for white vs. bright green is greater than for white vs. black.

In [1]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie1976

In [2]:
lab_from_rgb = lambda r, g, b: convert_color(sRGBColor(r, g, b), LabColor)

white = lab_from_rgb(1, 1, 1)
green = lab_from_rgb(0, 1, 0)
black = lab_from_rgb(0, 0, 0)

In [3]:
delta_e_cie1976(white, green)

120.40810187011418

In [4]:
delta_e_cie1976(white, black)

99.99998490087933